<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex14notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex14notebookC

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo14.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# 機械学習ライブラリ scikit-learn のいろいろ
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

# 「1000種類の物体を識別するニューラルネットを動かしてみよう」のためのほげ
import pickle
import torch
from torchvision import models, transforms
from PIL import Image
import json

---
## 機械学習のデモ
---

この授業で学んだ内容の延長上には様々な話がありますが，ここでは，その一つの方向である **機械学習** についてのデモを実行してみましょう．
機械学習については，3年1Q,2Qの「機械学習I/II」で学ぶことができます．

「機械学習I/II」のページ https://www-tlab.math.ryukoku.ac.jp/wiki/?ML


---
### ロジスティック回帰と階層型ニューラルネットによる手書き数字識別



多変量解析の授業で学んだ **判別分析** は，データをいくつかのクラスに分類する手法でした．
機械学習の授業では，同様の問題を扱う新たな手法をいくつか学びます．ここでは，それらの手法の中から，「ロジスティック回帰」と「階層型ニューラルネット」という二つをとりあげて，手書き数字識別の実験をやってみることにします．

#### データの準備


In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
X_train = rv['datL'].astype(float)
Y_train = rv['labL']
X_test = rv['datT'].astype(float)
Y_test = rv['labT']
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

K = 10 # クラス数
N_train, D = X_train.shape
N_test, _  = X_test.shape

この授業でも何度か扱った手書き数字画像のデータです．

In [ ]:
# データを画像として表示するための関数
#
def displayImage(ax, data, nx, ny, nrow=28, ncol=28, gap=4):

    assert data.shape[0] == nx*ny
    assert data.shape[1] == nrow*ncol

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = data[iy*nx+ix].reshape((nrow, ncol))

    ax.axis('off')
    ax.imshow(img, cmap='gray')

In [ ]:
# 学習データの最初の nx * ny 個を画像として表示
nx, ny = 10, 5
fig, ax = plt.subplots(figsize=(8, 4))
displayImage(ax, X_train[:ny*nx], nx, ny)
plt.show()

# それらのクラスラベルを表示
for iy in range(ny):
    print(Y_train[iy*nx:(iy+1)*nx])

In [ ]:
# データの各要素を平均 0 分散1 にスケーリング
scaler = StandardScaler()
XX_train = scaler.fit_transform(X_train)
XX_test  = scaler.transform(X_test)

#### ロジスティック回帰

判別分析では，クラスの正解を知っているデータ（学習データ）を用いてモデル（正規分布）のパラメータを推定し，得られたモデルを使ってクラス未知のデータ（テストデータ）のクラスを予測する，ということをしていました．
ロジスティック回帰や階層型ニューラルネットも，同様のことをします．ただし，これらの手法ではパラメータの推定が難しいので，パラメータに適当な初期値を設定して，データにうまく当てはまるように修正していくことを何度も繰り返します（どうやって？という話は「機械学習I/II」で学びます．ここでは気にしなくて構いません）．

In [ ]:
# ロジスティック回帰モデルのインスタンスを生成
model = LogisticRegression(max_iter=500)
print(model)

試しに，ロジスティック回帰モデルに手書き数字画像（学習データ 5000 枚）を 500 回学習させてみます．

In [ ]:
# 学習データを用いてモデルのパラメータを調整（学習）
model.fit(XX_train, Y_train)

学習したモデルにクラスを予測させて，何個正解したか求めてみると，こんなんなります．

In [ ]:
# 学習データのクラスを予測させてみる
Yt_train = model.predict(XX_train)
nc_train = np.sum(Yt_train == Y_train) # 正解数

# テストデータのクラスを予測させてみる
Yt_test = model.predict(XX_test)
nc_test = np.sum(Yt_test == Y_test) # 正解数

# 結果を表示
print(f'学習データ: {nc_train}/{len(Y_train)} = {nc_train/len(Y_train)}')
print(f'テストデータ: {nc_test}/{len(Y_test)} = {nc_test/len(Y_test)}')

#### 階層型ニューラルネット

「ニューラルネット」は，現在の AI 技術の中心となっている機械学習モデルです．
どんなものかの説明は省略しますが，たくさんのパラメータを含んだ複雑なモデルです．ロジスティック回帰と同様に，パラメータの推定を何度も繰り返して徐々に学習していきます．

In [ ]:
# 階層型ニューラルネットのインスタンスを生成
model = MLPClassifier(hidden_layer_sizes=(200, 200,), activation='relu', verbose=1)
print(model)

次のセルを実行すると，学習データ 5000 個を使って学習します．
`Iteration` は1回の学習（5000個のデータのそれぞれを使ってパラメータを推定する処理）を表し，`loss` はその学習を終えた時点での「損失」（正解とのずれの指標，小さいほど良い）の値を表します．

繰り返すごとに損失が小さくなり，学習が進んでいくのがわかるでしょう．

In [ ]:
# 学習データを用いてモデルのパラメータを調整（学習）
model.fit(XX_train, Y_train)

学習したモデルにクラスを予測させて，何個正解したか求めてみると，こんなんなります．

In [ ]:
# 学習データのクラスを予測させてみる
Yt_train = model.predict(XX_train)
nc_train = np.sum(Yt_train == Y_train) # 正解数

# テストデータのクラスを予測させてみる
Yt_test = model.predict(XX_test)
nc_test = np.sum(Yt_test == Y_test) # 正解数

# 結果を表示
print(f'学習データ: {nc_train}/{len(Y_train)} = {nc_train/len(Y_train)}')
print(f'テストデータ: {nc_test}/{len(Y_test)} = {nc_test/len(Y_test)}')

ロジスティック回帰よりも複雑なモデルであるため，ここではより高い予測精度が得られています．
ただし，実際にはどんな場合でも複雑なモデルの方が良いというわけではありません．その辺のことは「機械学習I/II」で学びます．

---
### 1000種類の物体を識別するニューラルネットを動かしてみよう

ニューラルネットワークを用いた画像識別の実験をやってみましょう． いろんなものが識別できた方が楽しいと思いますので，1000種類の物体を識別させてみます． ただし，そういうことができるニューラルネットワークを一から学習させようとすると大変（↑ののりで実行したら1週間以上かかかります）ですので，ここではすでに学習済みのものを利用することにします．

#### 準備

まずは実験に使う画像データを準備します．

**注意: 以下の画像をこの実験以外の目的で使用してはいけません**

In [ ]:
!wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/animalphoto.pickle
with open('animalphoto.pickle', 'rb') as f:
    hoge = pickle.load(f)

imgList = []
for d in hoge:
    img = Image.frombytes(**d)
    imgList.append(img)
    fig, ax = plt.subplots(1)
    ax.imshow(img)
    ax.axis('off')
    plt.show()

1000個あるクラスの番号とクラスラベルの対応表を作成します．

In [ ]:
# ImageNet クラスラベルを表すJSONファイルを入手
!wget -nc https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
class_index = json.load(open('imagenet_class_index.json', 'r'))

K = len(class_index)
labels = {}
for ik, key in enumerate(class_index.keys()):
    labels[ik] = f'{class_index[key][0]} {class_index[key][1]}'
    if 276 <= ik < 300: 
        print(ik, labels[ik])

上記の出力は，1000クラスのうちの一部のものの名前を表します．「ネコ」みたいなものも1クラスではなく，281番 'tabby'（ぶち柄の猫），282番 'tiger_cat'（トラ縞の猫）等々のように分かれています．

学習済みのニューラルネットワークのパラメータを入手します．
規模の大きいネットワークでパラメータがたくさんありますので，読み込みに少し時間がかかります．

In [ ]:
vgg16 = models.vgg16(pretrained=True)
vgg16.eval()
print(vgg16)

上記のセルを実行すると，次のことが行われます．
1. ニューラルネットワークの学習モデルを作成する．
1. 学習済みのネットワークパラメータの値を読み込んでこのネットワークのパラメータに設定する．
1. ネットワークモデルの構造を表示する．

参考:

ここで用いているのは VGG16 というニューラルネットワークモデルです．イギリスオックスフォード大学の [Visual Geometry Group(VGG)](https://www.robots.ox.ac.uk/~vgg/) という研究グループが作った VGG-net というニューラルネットワークモデルのうちの，層が16層あるものです．
VGG-net は，2014年に行われた [ILSVRC2014](https://www.image-net.org/challenges/LSVRC/2014/) という画像識別のコンペティションで世界第2位となったニューラルネットワークモデルです．
ILSVRC2014では，[ImageNet](https://www.image-net.org/) という大規模な画像データセットの中から選ばれた 1000 種類の物体の画像約120万枚を学習データとしていました．


画像をネットワークに入力できる形式に加工する処理を定義しておきます．


In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

#### 実験1

上記のサンプル画像たちを VGG16 に識別させてみましょう．


In [ ]:
#@title 実験1
#@markdown 以下で 0 から 3 までの整数を選ぶと4つのサンプル画像の中から一つを選ぶことができます．
i = 0 #@param [0, 1, 2, 3] {type: 'raw'}
img = imgList[i]

# 画像を表示
fig, ax = plt.subplots(1)
ax.imshow(img)
ax.axis('off')
plt.show()

# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(imgList[i]), axis=0)
Y = vgg16(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')

上記の画像の下の出力は，1000個のクラスの中で，ネットワークの出力の値が大きかったもの上位5位までの，出力の値とクラス名を表します．
出力の値は，そのクラスと判定することの「確信度」と解釈できます．

#### 実験2

自分で用意した画像でも実験してみましょう．

- 上記では動物の例ばかりですが，1000のクラスの中にはそれ以外にも様々なものがあります（人間のクラスはありません）
- 面白い／不思議な結果が得られたら takataka に見せてくれると喜びます



(1) まずは，ウェブ等で適当な画像を探して，この Colab notebook を実行しているコンピュータへ保存しましょう．ファイル名が長かったり日本語を含んでいる場合は，短い名前に変更しておくのがよいです．

(2) 以下のセルを実行して，ファイルをアップロードします．

In [ ]:
# Colab へファイルをアップロード
try:
    from google.colab import files
    rv = files.upload()
except:
    print('このコードは Colab 以外の環境では実行できないよ．')

(3) 以下のセルの1行目のファイル名を上記でアップロードしたものに変えて実行しましょう．アルファベット大文字小文字の区別もありますので注意．うまくいけば画像が表示されるはずです．

In [ ]:
myimg = Image.open('hoge.jpg') # ファイル名を変更しましょう
fig, ax = plt.subplots(1)
ax.imshow(myimg)
ax.axis('off')
plt.show()

(4) 以下のセルを実行すれば結果が表示されます．

In [ ]:
# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(myimg), axis=0)
Y = vgg16(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')